In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../inc')


In [3]:
# Load raw lodging list
df = pd.read_csv('../DABA/서울시 관광숙박업 인허가 데이터.csv', encoding='cp949')
#df = pd.read_csv('../DABA/서울시 숙박업 인허가 정보.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 61 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   개방자치단체코드    610 non-null    int64  
 1   관리번호        610 non-null    object 
 2   인허가일자       610 non-null    object 
 3   인허가취소일자     1 non-null      float64
 4   영업상태코드      610 non-null    int64  
 5   영업상태명       610 non-null    object 
 6   상세영업상태코드    610 non-null    int64  
 7   상세영업상태명     610 non-null    object 
 8   폐업일자        136 non-null    object 
 9   휴업시작일자      9 non-null      object 
 10  휴업종료일자      9 non-null      object 
 11  재개업일자       0 non-null      float64
 12  전화번호        508 non-null    object 
 13  소재지면적       0 non-null      float64
 14  소재지우편번호     236 non-null    object 
 15  지번주소        610 non-null    object 
 16  도로명주소       602 non-null    object 
 17  도로명우편번호     516 non-null    float64
 18  사업장명        610 non-null    object 
 19  최종수정일자      610 non-null    o

In [9]:
# select item
sel_colum = [2,3,4,8,9,10,11,15,16,17,18,21,23,24,32,35,46,59,60]
sel_df = df.iloc[:,sel_colum]
sel_df.info()

if pd.notnull(sel_df.지번주소[137]): print('111')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   인허가일자     610 non-null    object 
 1   인허가취소일자   1 non-null      float64
 2   영업상태코드    610 non-null    int64  
 3   폐업일자      136 non-null    object 
 4   휴업시작일자    9 non-null      object 
 5   휴업종료일자    9 non-null      object 
 6   재개업일자     0 non-null      float64
 7   지번주소      610 non-null    object 
 8   도로명주소     602 non-null    object 
 9   도로명우편번호   516 non-null    float64
 10  사업장명      610 non-null    object 
 11  데이터갱신일자   610 non-null    object 
 12  좌표정보(X)   603 non-null    float64
 13  좌표정보(Y)   603 non-null    float64
 14  건물용도명     209 non-null    object 
 15  객실수       307 non-null    float64
 16  시설면적      358 non-null    float64
 17  시설규모      358 non-null    float64
 18  관광숙박업상세명  319 non-null    object 
dtypes: float64(8), int64(1), object(10)
memory usage: 90.7+ KB
111


In [12]:
# .. 카카오API로 위경도 삽입, pyproj로 시도하였으나 값이 약간 다름

import requests # 온라인으로 보내거나 받아올 때 사용, 파이썬으로 웹사이트 연동할 때
from urllib.parse import quote # url에서 한글 코드를 변환
import numpy as np


# API 키 가져오기
filename = '/Users/mankikim/Desktop/프로젝트/더조은/API/kakao_API.txt'
with open(filename) as f:
    api_key = f.read().strip()
    
len(api_key)  # 32글자가 되야됨, 파일키 정상 문자열개수
     
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list, err_list = [], [], []

for i in sel_df.index:
#for i in [37]:
    print(f'{i} , {sel_df.사업장명[i]}')

    addr = np.nan
    
    if pd.notnull(sel_df.지번주소[i]): addr = sel_df.지번주소[i]    # 관광데이터는 지번주소가 무조건 있음
    #if pd.notnull(sel_df.도로명주소[i]): addr = sel_df.도로명주소[i]
    #if pd.isna(addr): print( f"{i}번째 {sel_df.사업장명[i]}은 주소가 아무것도 없습니다, 패스")

    temp = quote(addr)

    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()

    print(result)

    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        lon_list.append(np.NAN)
        lat_list.append(np.NAN)
        err_list.append((i,sel_df.지번주소[i]))

    
sel_df['위도'] = lat_list
sel_df['경도'] = lon_list

sel_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mankikim/Desktop/프로젝트/더조은/API/카카오API.txt'

In [27]:
print(err_list)
print(len(err_list))
sel_df.info()


[(0, '서울특별시 중구 인현동1가 158'), (21, '서울특별시 서초구 서초동 1317-9 ,10호 2층~6층'), (39, '서울특별시 중구 오장동 139-20 삼덕빌딩 4,5,6층'), (42, '서울특별시 중구 필동1가 43-1'), (44, '서울특별시 중구 북창동 13-2'), (58, '서울특별시 중구 인현동1가 94-1'), (64, '서울특별시 강동구 천호동 328-10'), (67, '서울특별시 중구 충무로2가 12-8'), (76, '서울특별시 종로구 창신동 460-1번지 '), (93, '서울특별시 중구 쌍림동 25-16'), (101, '서울특별시 성동구 도선동 117-0'), (103, '서울특별시 성동구 도선동 115-0'), (109, '서울특별시 종로구 낙원동 283-14'), (110, '서울특별시 중구 필동1가 43-1'), (123, '서울특별시 양천구 신월동 83-4'), (132, '서울특별시 종로구 익선동 53'), (154, '서울특별시 강남구 역삼동 678-14 에이치에비뉴호텔 역삼'), (155, '서울특별시 성북구 동선동3가 41-1'), (164, '서울특별시 은평구 대조동 15-55번지 '), (169, '서울특별시 중구 오장동 139-6'), (170, '서울특별시 중구 태평로2가 68-6'), (178, '서울특별시 중구 을지로6가 18-106번지 '), (189, '서울특별시 중구 인현동1가 158'), (190, '서울특별시 중구 명동2가 4-2'), (201, '서울특별시 중구 남대문로5가 624'), (205, '서울특별시 관악구 신림동 1423-6'), (209, '서울특별시 중구 회현동1가 194-15 4~18층'), (214, '서울특별시 은평구 대조동 15-49'), (232, '서울특별시 중랑구 상봉동 89'), (234, '서울특별시 중구 신당동 349-110번지 '), (242, '서울특별시 종로구 무악동 45-14번지 '), (248, '서울특별시 광진구 중곡동 647-16'),

In [28]:
# 사업장명 컬럼 맨 앞으로 
sel_df2 = sel_df[['사업장명','지번주소','도로명주소','위도','경도','인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자','데이터갱신일자','업태구분명','한실수','양실수','욕실수']]
sel_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6664 entries, 0 to 6663
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     6664 non-null   object 
 1   지번주소     6653 non-null   object 
 2   도로명주소    3986 non-null   object 
 3   위도       5287 non-null   float64
 4   경도       5287 non-null   float64
 5   인허가일자    6664 non-null   object 
 6   인허가취소일자  0 non-null      float64
 7   영업상태코드   6664 non-null   int64  
 8   폐업일자     4173 non-null   object 
 9   휴업시작일자   0 non-null      float64
 10  휴업종료일자   0 non-null      float64
 11  재개업일자    0 non-null      float64
 12  데이터갱신일자  6664 non-null   object 
 13  업태구분명    6664 non-null   object 
 14  한실수      5434 non-null   float64
 15  양실수      5347 non-null   float64
 16  욕실수      4813 non-null   float64
dtypes: float64(9), int64(1), object(7)
memory usage: 885.2+ KB


In [29]:
sel_df2.to_csv("../DAIN/crw_list_extd.txt")

In [30]:
err_df = pd.DataFrame(err_list, columns=['index','위경도 missing'])
err_df.to_csv("../DAIN/err_list_extd.txt")